In [ ]:

## Forma mais direta
import sqlite3
connection = sqlite3.connect("pizza_app.sqlite")

## SQLite

### "Conectando" ao database

Por não se tratar de um banco cliente-servidor, a conexão ao banco é na verdade o acesso ao arquivo (ou criação dele) que contém as informações do banco. A conexão pode ser feita com o seguinte código:

```python
import sqlite3
connection = sqlite3.connect(path)
```
O parâmetro `path` em geral é o nome do arquivo onde os registros estão/estarão armazenados. Mas também pode ser a string `:memory:` caso se deseje criar um banco provisório em memória, o qual deixará de existir quando a conexão for fechada.

Precisamos armazenar o [objeto Connection](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection) retornado pela chamada para os próximos passos.



In [ ]:
## Forma mais direta
import sqlite3
connection = sqlite3.connect("pizza_app.sqlite")

### Executando queries

O objeto `connection`, retornado pela conexão que acabamos de criar, possui um método `cursor()` que cria e retorna um novo objeto `Cursor`. O objeto `Cursor` é um intermediário para enviar comandos SQL para o banco de dados e recuperar resultados (veja a [documentação](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor)). Principais métodos:

* `execute`: Executa uma instrução SQL, recebida como string.
```python
cursor.execute("INSERT INTO data VALUES ('queijo', 4.5);")
```
    * Opcionalmente pode-se adicionar placeholders à string e preenchê-los em `parameters`. [Entenda como adicionar placeholders em queries SQL](https://docs.python.org/3/library/sqlite3.html#how-to-use-placeholders-to-bind-values-in-sql-queries).
```python
execute(sql: str, parameters: tuple = ())
# exemplo com placeholders nomeados:
dic = ({'ing': 'queijo', 'preco': 4.5})
cursor.execute("INSERT INTO data VALUES(:ing, :preco)", dic)
```
* `executemany`: Repete a execução da instrução `sql` dada com cada um dos elementos de `parameters`.
```python
executemany(sql: str, parameters: Sequence[tuple])
# exemplo com ? placeholders
rows = [("queijo", 4.5), ("azeitona", 1)]
cursor.executemany("INSERT INTO data VALUES(?, ?)", rows)
```
* `executescript`: Permite executar scripts sql.
```python
executescript(sql_script: str)
#exemplo
cursor.executescript("""
    BEGIN;
    CREATE TABLE person(firstname, lastname, age);
    CREATE TABLE book(title, author, published);
    COMMIT;
""")
```

> **`rowcount`**: Atributo da classe `Cursor` com o número de linhas afetadas por instruções `INSERT`, `UPDATE`, `DELETE` ou `REPLACE`. É atualizado pelos méetodos `execute()` e `executemany()`. Se nenhuma linha foi afetada pela operação, é `-1`.

> ❗ Instrução `INSERT` ❗ abre uma [transação](https://docs.python.org/3/library/sqlite3.html#sqlite3-controlling-transactions) que fica pendente até que se invoque `connection.commit()`. Só então as atualizações são salvas no banco de dados. Caso nenhuma transação esteja pendente, o commit não faz nada.


O código a seguir apresenta uma função mais robusta para executar queries, incluindo o tratamento de erros mapeado pelo sqlite Error:
* [`sqlite3.Error`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Error): Classe base para todas as exceções dessa biblioteca (herda da python `Exception`). Exceções lançadas pela própria biblioteca tem dois atributos:
    * `sqlite_errorcode`: Código de erro da [API SQLite](https://sqlite.org/rescode.html).
    * `sqlite_errorname`: Nome simbólico do erro.

In [ ]:
from sqlite3 import Error

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)

        # commit necessário para alterações no banco
        connection.commit() ###

        print(f"Query executada.")
        if cursor.rowcount != -1:
            print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}'")

In [ ]:
# Cria a tabela produto #
create_produto_table = \
"""CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
);"""

execute_query(connection, create_produto_table)
#########################

# Cria a tabela pedido #
create_pedido_table = \
"""CREATE TABLE pedido (
    id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT
);"""

execute_query(connection, create_pedido_table)
#########################

# Cria a tabela item_pedido #
create_item_pedido_table = \
"""CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
);"""
execute_query(connection, create_item_pedido_table)
#########################

Query executada.
Query executada.
Query executada.


In [ ]:
## Inserindo registros manualmente

# Inserindo produto #
insert_produto = \
"""INSERT INTO
produto (tipo, desc_item, vl_preco)
VALUES
('ingrediente', 'camarão', 6),
('massa', 'tradicional', 9.25),
('borda', 'tradicional', 0),
('queijo', 'muçarela', 4),
('bebida', 'refrigerante', 5);
"""
execute_query(connection, insert_produto)
######################

# Inserindo pedido
insert_pedido = \
"""INSERT INTO
pedido (dt_pedido, fl_ketchup, desc_uf, txt_recado)
VALUES
('2023-06-01', TRUE, 'MG', 'Capricha no queijo!');
"""
execute_query(connection, insert_pedido)
######################

Query executada.
5 linha(s) afetadas
Query executada.
1 linha(s) afetadas


A célula a seguir é um exemplo de uso do `executemany`, uma das formas de automatizar alterações em grande volume.

In [ ]:
# Inserindo item_pedido
itens = (
    {'id_pedido': 1, 'id_produto': 2, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 3, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 1, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 4, 'qtd': 2},
    {'id_pedido': 1, 'id_produto': 5, 'qtd': 3}
)

insert_item_pedido = \
"""INSERT INTO item_pedido (id_pedido, id_produto, quantidade)
VALUES(:id_pedido, :id_produto, :qtd);"""

cursor = connection.cursor()
cursor.executemany(insert_item_pedido, itens)
connection.commit() # necessário para inserções
cursor.close()

### Executando queries de leitura

Após executar uma instrução de consulta ao banco através do objeto `Cursor`, o resultado da consulta pode ser recuperado invocando um dos seguintes métodos do cursor:

* `fetchone`: Retorna a próxima linha do resultado da consulta. Cada nova chamada a fetchone() retorna a linha subsequente (como um cursor avançando após ler uma linha). Seu retorno é uma única tupla com os dados do registro.
```python
fetchone() -> tuple
```
* `fetchall`: Retorna uma lista de tuplas com todas as linhas resultantes da consulta. Cada tupla é um registro do banco.
```python
fetchall() -> list[tuple]
```
* `fetchmany`: Retorna o próximo conjunto de linhas da consulta, de acordo com parâmetro `size`. Se `size` não for especificado, o valor padrão é `cursor.arraysize`. Também retorna uma lista de tuplas.
```python
fetchmany(size: int = cur.arraysize) -> list[tuple]
```

In [ ]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

In [ ]:
tabela = 'produto'
query = f"SELECT * FROM {tabela}"
resultado = execute_read_query(connection, query)

print(f"Tabela: {tabela}")
for res in resultado:
    print(res)

Tabela: produto
(1, 'ingrediente', 'camarão', 6)
(2, 'massa', 'tradicional', 9.25)
(3, 'borda', 'tradicional', 0)
(4, 'queijo', 'muçarela', 4)
(5, 'bebida', 'refrigerante', 5)


Todo banco de dados SQLite tem uma "tabela de esquema" que armazena o esquema desse banco de dados. A tabela é representada a seguir e você pode ver mais detalhes sobre ela na [documentação da API](https://www.sqlite.org/schematab.html).
```
CREATE TABLE sqlite_schema(
  type text,
  name text,
  tbl_name text,
  rootpage integer,
  sql text
);
```

Vamos criar uma query para ler o nome de todas as tabelas registradas no esquema:
```sql
SELECT name FROM sqlite_schema WHERE type='table';
```

Note no resultado que além das tabelas que nós criamos, também existe uma tabela extra:
* [`sqlite_sequence`](https://www.sqlite.org/fileformat2.html#seqtab): É uma tabela interna para implementar `AUTOINCREMENT`. É criada automaticamente sempre que qualquer tabela comum com uma chave primária inteira AUTOINCREMENT é criada. Uma vez criada, existirá na tabela `sqlite_schema` para sempre; não pode ser descartada.

In [ ]:
select_table_names = \
"SELECT name FROM sqlite_schema WHERE type='table';"
tables = execute_read_query(connection, select_table_names)
print(tables, '\n')

for table in tables:
    select_all = f"SELECT * FROM {table[0]}"
    res = execute_read_query(connection, select_all)
    print(f"{table[0]}: {res}")

[('produto',), ('sqlite_sequence',), ('pedido',), ('item_pedido',)] 

produto: [(1, 'ingrediente', 'camarão', 6), (2, 'massa', 'tradicional', 9.25), (3, 'borda', 'tradicional', 0), (4, 'queijo', 'muçarela', 4), (5, 'bebida', 'refrigerante', 5)]
sqlite_sequence: [('produto', 5), ('pedido', 1)]
pedido: [(1, '2023-06-01', 1, 'MG', 'Capricha no queijo!')]
item_pedido: [(1, 2, 1), (1, 3, 1), (1, 1, 1), (1, 4, 2), (1, 5, 3)]


### Dropando...

A seguir vamos dropar todos os registros de todas as tabelas que criamos manualmente, para a seguir conhecer uma maneira mais interessante de carregamento de dados. Não façam isso em ~casa~ projetos reais 😆

In [ ]:
execute_query(connection, "DELETE FROM item_pedido;")
execute_query(connection, "DELETE FROM pedido;")
execute_query(connection, "DELETE FROM produto;")

Query executada.
5 linha(s) afetadas
Query executada.
1 linha(s) afetadas
Query executada.
5 linha(s) afetadas


### Carregando arquivos CSV com Pandas 🐼

Antes de tudo, baixe a versão modificada dos dados do Pizza Query.

In [ ]:
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/item_pedido.csv
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/pedido.csv
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/produto.csv

--2025-07-09 15:25:28--  https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/item_pedido.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103557 (101K) [text/plain]
Saving to: ‘item_pedido.csv’

item_pedido.csv     100%[===================>] 101.13K  --.-KB/s    in 0.02s   

2025-07-09 15:25:28 (5.39 MB/s) - ‘item_pedido.csv’ saved [103557/103557]

--2025-07-09 15:25:28--  https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/pedido.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting

Segundo a [documentação](https://pandas.pydata.org/), o Pandas é "uma ferramenta de **análise e manipulação de dados** de código aberto, rápida, poderosa, flexível e fácil de usar". A biblioteca basicamente cria tabelas parecidas com planilhas Excel, chamadas de **`DataFrame`**, com uma interface orientada a objetos e uma biblioteca gigante de funções pra manipular os dados.

In [ ]:
import pandas as pd
df_pedido = pd.read_csv(f'pedido.csv')
display(df_pedido.head())

,id_pedido,dt_pedido,fl_ketchup,desc_uf,txt_recado
0,0,2023-05-11,NaN,GO,NaN
1,1,2023-05-11,NaN,PR,Aquela pizza perfeita! :-D
2,2,2023-05-11,NaN,SP,Muito obrigado!!
3,3,2023-05-11,NaN,SP,NaN
4,4,2023-05-11,NaN,RS,Capricha no peperoni


Para carregar tabelas em bancos sqlite, o Pandas entra em cena com o método `.to_sql()`. Ele facilita a exportação de DataFrames para bancos de dados SQL [(veja a documentação)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html). Esse método é compatível tanto com conexões do sqlite3 quanto os bancos suportados pelo SQLAlchemy. A seguir está um exemplo de chamada com os principais parâmetros.

```python
df.to_sql(table_name, connection, if_exists='fail', index=False)
```

O parâmetro `if_exists` controla o comportamento caso a tabela já exista no banco de dados, podendo assumir os seguintes valores:

* `'fail'`: Gera um erro se a tabela já existir (padrão).
* `'replace'`: Remove a tabela existente antes de inserir os dados.
* `'append'`: Adiciona os dados ao final da tabela existente.

In [ ]:
df_pedido.to_sql('pedido', connection, if_exists='append', index=False)

count_rows = "SELECT COUNT(id_pedido) as count_id FROM pedido;"
print(execute_read_query(connection, count_rows))

select_all = f"SELECT * FROM pedido WHERE id_pedido < 5;"
execute_read_query(connection, select_all)

[(1106,)]


[(0, '2023-05-11', None, 'GO', None),
 (1, '2023-05-11', None, 'PR', 'Aquela pizza perfeita! :-D'),
 (2, '2023-05-11', None, 'SP', 'Muito obrigado!!'),
 (3, '2023-05-11', None, 'SP', None),
 (4, '2023-05-11', None, 'RS', 'Capricha no peperoni')]

Mas cuidado com coisas automáticas! Ao criar tabelas com o `to_sql` (por exemplo definindo `if_exists='replace'`) os tipos de dados serão inferidos automaticamente e podem não ser tipos desejáveis para as regras do banco.

Preste atenção na tabela Pedido que criamos manualmente. Em seguida descomente a linha da célula a seguir e execute novamente. Note que os tipos de dados escolhidos tendem a ser mais genéricos, já que não existe uma compreensão de modelagem do banco.

In [ ]:
# df_pedido.to_sql('pedido', connection, if_exists='replace', index=False)
res = execute_read_query(connection, "SELECT sql FROM sqlite_schema")
for r in res:
    print(r[0])

CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
)
CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE pedido (
    id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT
)
CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
)
None


In [ ]:
df_pedido.to_sql('pedido', connection, if_exists='replace', index=False)
res = execute_read_query(connection, "SELECT sql FROM sqlite_schema")
for r in res:
    print(r[0])

CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
)
CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
)
None
CREATE TABLE "pedido" (
"id_pedido" INTEGER,
  "dt_pedido" TEXT,
  "fl_ketchup" INTEGER,
  "desc_uf" TEXT,
  "txt_recado" TEXT
)


O Pandas é uma interface de comunicação com bancos de dados SQL não só na criação e/ou preenchimento de tabelas, como também permitindo executar queries. A vantagem nesse caso é que o retorno da query é estruturado como um objeto tipo DataFrame que pode ser trabalhando posteriormente no Python

In [ ]:
query="""
SELECT desc_uf, COUNT(*) as count_pedidos
FROM pedido
GROUP BY desc_uf
ORDER BY count_pedidos DESC
LIMIT 5
"""
pd.read_sql_query(query, connection)

,descUF,count_pedidos
0,SP,395
1,RJ,103
2,MG,92
3,PR,76
4,RS,50
